In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

print("=== Loading preprocessed data ===")
# Load the data (recreate since we need it in this script)
mrna_data = pd.read_csv("/workdir/KIRC_mRNA_top_column_cleaned.csv")
mrna_transposed = mrna_data.set_index('unnamed_0').T
survival_data = pd.read_csv("/workdir/survival_KIRC.csv")

# Get matched samples
matched_samples = list(set(mrna_transposed.index).intersection(set(survival_data['sample_name'])))
matched_expression_data = mrna_transposed.loc[matched_samples].copy()
matched_survival_data = survival_data[survival_data['sample_name'].isin(matched_samples)].copy()
matched_survival_data = matched_survival_data.set_index('sample_name').loc[matched_samples].reset_index()

# Calculate variance and get initial features
gene_variances = matched_expression_data.var(axis=0).sort_values(ascending=False)
n_initial_features = min(1000, len(gene_variances))
initial_features = gene_variances.head(n_initial_features).index.tolist()

print(f"Working with {len(matched_samples)} patients and {len(initial_features)} initial features")

print("\n=== Step 5-7: Iterative optimization for survival separation ===")

# Function to evaluate clustering quality based on survival separation
def evaluate_clustering(expression_data, features, survival_data, random_state=42):
    """Evaluate clustering quality based on survival separation"""
    try:
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(expression_data[features])
        
        # Perform K-means clustering
        kmeans = KMeans(n_clusters=2, random_state=random_state, n_init=10)
        cluster_labels = kmeans.fit_predict(X_scaled)
        
        # Get survival data for each cluster
        cluster_0_indices = np.where(cluster_labels == 0)[0]
        cluster_1_indices = np.where(cluster_labels == 1)[0]
        
        if len(cluster_0_indices) < 5 or len(cluster_1_indices) < 5:
            return None, None, float('inf')  # Invalid clustering
        
        # Extract survival data for each cluster
        times_0 = survival_data.iloc[cluster_0_indices]['survival_times']
        events_0 = survival_data.iloc[cluster_0_indices]['event_observed']
        times_1 = survival_data.iloc[cluster_1_indices]['survival_times']
        events_1 = survival_data.iloc[cluster_1_indices]['event_observed']
        
        # Perform log-rank test
        results = logrank_test(times_0, times_1, event_observed_A=events_0, event_observed_B=events_1)
        p_value = results.p_value
        
        return cluster_labels, kmeans, p_value
        
    except Exception as e:
        print(f"Error in clustering evaluation: {e}")
        return None, None, float('inf')

# Initial clustering
print("Performing initial clustering...")
initial_clusters, initial_model, initial_p_value = evaluate_clustering(
    matched_expression_data, initial_features, matched_survival_data
)

print(f"Initial p-value: {initial_p_value:.6f}")

# Initialize optimization variables
best_features = initial_features.copy()
best_clusters = initial_clusters.copy()
best_p_value = initial_p_value
best_model = initial_model
iteration_results = []

print("\nStarting iterative optimization...")

# Iterative optimization (simplified approach)
max_iterations = 5  # Reduced for efficiency
for iteration in range(max_iterations):
    print(f"\nIteration {iteration + 1}/{max_iterations}")
    
    # Current feature set size
    current_n_features = len(best_features)
    step_size = max(10, current_n_features // 20)  # Remove/add 5% of features
    
    # Get unused features sorted by variance
    unused_features = [gene for gene in gene_variances.index if gene not in best_features]
    unused_top = unused_features[:step_size * 2]  # Get top unused features
    
    if len(unused_top) == 0:
        print("No more unused features available")
        break
    
    # Try different feature modifications
    modifications = [
        ("add_high_var", best_features + unused_top[:step_size]),
        ("replace_low_var", best_features[:-step_size] + unused_top[:step_size]),
    ]
    
    for mod_name, new_features in modifications:
        if len(new_features) < 50:  # Minimum feature threshold
            continue
            
        # Evaluate new feature set
        new_clusters, new_model, new_p_value = evaluate_clustering(
            matched_expression_data, new_features, matched_survival_data
        )
        
        iteration_results.append({
            'iteration': iteration + 1,
            'modification': mod_name,
            'n_features': len(new_features),
            'p_value': new_p_value
        })
        
        print(f"  {mod_name}: p-value = {new_p_value:.6f} (features: {len(new_features)})")
        
        # Update if improvement found
        if new_p_value < best_p_value and new_clusters is not None:
            best_features = new_features.copy()
            best_clusters = new_clusters.copy()
            best_p_value = new_p_value
            best_model = new_model
            print(f"  *** New best p-value: {best_p_value:.6f} ***")

print(f"\nOptimization completed!")
print(f"Best p-value: {best_p_value:.6f}")
print(f"Best feature count: {len(best_features)}")

# Final clustering with best features
print("\n=== Step 8: Final clustering and survival analysis ===")

# Standardize final features
scaler = StandardScaler()
X_final = scaler.fit_transform(matched_expression_data[best_features])

# Final clustering
final_clusters = best_clusters.copy()

# Add cluster labels to survival data
matched_survival_data['cluster'] = final_clusters
cluster_0_data = matched_survival_data[matched_survival_data['cluster'] == 0]
cluster_1_data = matched_survival_data[matched_survival_data['cluster'] == 1]

print(f"Cluster 0: {len(cluster_0_data)} patients")
print(f"Cluster 1: {len(cluster_1_data)} patients")

# Kaplan-Meier analysis
kmf_0 = KaplanMeierFitter()
kmf_1 = KaplanMeierFitter()

kmf_0.fit(cluster_0_data['survival_times'], cluster_0_data['event_observed'], label='Cluster 0')
kmf_1.fit(cluster_1_data['survival_times'], cluster_1_data['event_observed'], label='Cluster 1')

# Log-rank test
logrank_result = logrank_test(
    cluster_0_data['survival_times'], cluster_1_data['survival_times'],
    event_observed_A=cluster_0_data['event_observed'], 
    event_observed_B=cluster_1_data['event_observed']
)

print(f"Log-rank test p-value: {logrank_result.p_value:.6f}")
print(f"Log-rank test statistic: {logrank_result.test_statistic:.4f}")

# Cox proportional hazards model
cox_data = matched_survival_data[['survival_times', 'event_observed', 'cluster']].copy()
cox_data.columns = ['duration', 'event', 'cluster']

cph = CoxPHFitter()
cph.fit(cox_data, duration_col='duration', event_col='event')

print("\nCox Proportional Hazards Results:")
print(cph.summary)

hazard_ratio = np.exp(cph.params_['cluster'])
ci_lower = np.exp(cph.confidence_intervals_.loc['cluster', 'coef lower 95%'])
ci_upper = np.exp(cph.confidence_intervals_.loc['cluster', 'coef upper 95%'])

print(f"Hazard Ratio: {hazard_ratio:.4f} (95% CI: {ci_lower:.4f} - {ci_upper:.4f})")

# Save intermediate results
print("\nSaving analysis results...")
results_summary = {
    'best_p_value': best_p_value,
    'n_best_features': len(best_features),
    'cluster_0_size': len(cluster_0_data),
    'cluster_1_size': len(cluster_1_data),
    'logrank_p_value': logrank_result.p_value,
    'hazard_ratio': hazard_ratio,
    'ci_lower': ci_lower,
    'ci_upper': ci_upper,
    'cluster_0_median_survival': cluster_0_data['survival_times'].median(),
    'cluster_1_median_survival': cluster_1_data['survival_times'].median()
}

pd.Series(results_summary).to_csv("/workdir/execution_outputs/analysis_summary.csv")
pd.DataFrame(iteration_results).to_csv("/workdir/execution_outputs/iteration_results.csv")

print("Clustering optimization completed successfully!")
